# pre-processing

In [ ]:
import re
from bs4 import BeautifulSoup
from markdown import markdown
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

def comment_filter(x):
    if x:
        # Returning text stripping out all uris and replace with a sentences
        text = re.sub('http\S+',"(Here is the link)",x)
        # remove and \n
        text = re.sub("\n","",text)
        #remove several space ,leading and trailing whitespace
        reg_str_space = '\s+'
        # several whitespaces -> one space
        text = re.sub(reg_str_space, ' ', text) 
         # remove leading and trailing whitespace
        text = text.strip()
        return text
    else:
        return ""

Mounted at /content/drive
/content/drive/MyDrive/1afyp/ml/new_dataset


In [ ]:
import numpy as np

name = "vali_title"
col = 'title' # 'title'
com = pd.read_csv("{}.csv".format(name))
com[col] = com[col].apply(lambda x: comment_filter(x))
#remove empty row
com = com.replace(to_replace='', value=np.nan).dropna()
com.to_csv("{}_clean.csv".format(name),index=False)

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
%cd /content/drive/MyDrive/1afyp/ml/new_dataset

Mounted at /content/drive
/content/drive/MyDrive/1afyp/ml/new_dataset


In [ ]:
print(len(pd.read_csv("{}.csv".format("vali_text_clean"))))#10%
print(len(pd.read_csv("{}.csv".format("test_text_clean"))))#20%
print(len(pd.read_csv("{}.csv".format("train_text_clean"))))#70%

# print(len(pd.read_csv("{}.csv".format("vali_title_clean"))))
# print(len(pd.read_csv("{}.csv".format("test_title_clean"))))
# print(len(pd.read_csv("{}.csv".format("train_title_clean"))))

4429
8846
30992
4490
8980
31428


In [ ]:
print("vali_text_clean",pd.read_csv("{}.csv".format("vali_text_clean")).head(5))
print()
print("test_text_clean",pd.read_csv("{}.csv".format("test_text_clean")).head(5))
print()
print("train_text_clean",pd.read_csv("{}.csv".format("train_text_clean")).head(5))
print()

# print("vali_title_clean",pd.read_csv("{}.csv".format("vali_title_clean")).head(5))
# print()
# print("test_title_clean",pd.read_csv("{}.csv".format("test_title_clean")).head(5))
# print()
# print("train_title_clean",pd.read_csv("{}.csv".format("train_title_clean")).head(5))
# print()

vali_text_clean                                                 text  label
0  Martin Shkreli is a creep and not just because...      0
1  TOKYO (Reuters) - Japanese police arrested thr...      1
2  It warms the heart to know people as important...      0
3  After Ed Gillespie lost the VA Governor s race...      0
4  WASHINGTON (Reuters) - Republican George W. Bu...      1

test_text_clean                                                 text  label
0  LONDON (Reuters) - British Foreign Secretary B...      1
1  WASHINGTON (Reuters) - The leadership of the l...      1
2  Is there a single person left on Robert Muelle...      0
3  Celebrated Congressional Black Caucus member, ...      0
4  WASHINGTON (Reuters) - Republicans in the U.S....      1

train_text_clean                                                 text  label
0  MANILA (Reuters) - Philippine security forces ...      1
1  Thank goodnesss Jeff Sessions is moving on fin...      0
2  Senator Elizabeth Warren must be doing whateve

# political fake news classification with news content with RoBerta Model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/1afyp/ml/new_dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/1afyp/ml/new_dataset


install transformers library for roberta model #hugging face

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install pip install datasets

In [ ]:
from datasets import load_dataset
dataset_text = load_dataset("csv", data_files={"train":  "train_text_clean.csv", "test": "test_text_clean.csv","vali":"vali_text_clean.csv"})

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating vali split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a3912541d1968984/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

model = roberta


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig

MODEL_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_name)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_name)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

metric : accuracy

In [ ]:
from transformers.optimization import AdamW
import numpy as np
from datasets import load_metric
from sklearn.metrics import accuracy_score,recall_score,precision_score
from transformers import EarlyStoppingCallback, IntervalStrategy
def custom_metrics(eval_pred):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision = metric1.compute(predictions=predictions, references=labels, average="micro")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="micro")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="micro")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}


optimizer = AdamW(model.parameters(), lr=1e-3, correct_bias=True)



from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


tokenizer: need to understand
**WordPiece tokenizer**
https://towardsdatascience.com/how-to-build-a-wordpiece-tokenizer-for-bert-f505d97dddbb#:~:text=BERT%20uses%20what%20is%20called,have%20multiple%20forms%20of%20words.

In [ ]:
dataset_text["train"]["text"][1]

'Thank goodnesss Jeff Sessions is moving on finding out what went on behind the scenes to acquire FISA warrants. In other words, what was NOT said in order to get the judge to approve the FISA warrant to spy:We can start with why law enforcement officials who submitted the application failed to note that the Clinton campaign and DNC funded the dossier. The application does note that the dossier was put together by a U.S. person with political motivations, but Republicans have argued that the application should have been more specific.Via Daily Caller: Attorney General Jeff Sessions said Sunday that the Department of Justice is investigating whether the FBI submitted accurate information about the infamous Steele dossier in order to obtain a surveillance warrant against former Trump campaign adviser Carter Page. Are you, sir, investigating the fact that the FBI used the dossier to get a wiretap against Trump associates and they did not tell the FISA court that the Democrats and Hillary 

training paramter:
weight_decay=0.1,evaluation_strategy = IntervalStrategy.STEPS,eval_steps = 5,save_steps = 5, num_train_epochs=100,load_best_model_at_end=True,metric_for_best_model = 'accuracy'
with early stopping

In [ ]:
tokenized_datasets = dataset_text.map(tokenize_function, batched=True)

from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="roberta_text_100_step", save_total_limit = 5,weight_decay=0.1,evaluation_strategy = IntervalStrategy.STEPS,eval_steps = 5,save_steps = 5, num_train_epochs=100,load_best_model_at_end=True,metric_for_best_model = 'accuracy')


from transformers import EarlyStoppingCallback, IntervalStrategy
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["vali"].shuffle(seed=42)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=custom_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

  0%|          | 0/31 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

stop at step 45

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 30992
  Num Epochs = 100
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 387400
  Number of trainable parameters = 124647170


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
5,No log,0.672889,0.525401,0.525401,0.525401,0.525401
10,No log,0.578938,0.904493,0.904493,0.904493,0.904493
15,No log,0.084068,0.991646,0.991646,0.991646,0.991646
20,No log,0.035560,0.992549,0.992549,0.992549,0.992549
25,No log,0.299355,0.945812,0.945812,0.945812,0.945812
30,No log,0.006963,0.999097,0.999097,0.999097,0.999097
35,No log,0.009372,0.998645,0.998645,0.998645,0.998645
40,No log,0.015502,0.997742,0.997742,0.997742,0.997742
45,No log,0.018157,0.997742,0.997742,0.997742,0.997742


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4429
  Batch size = 8
<ipython-input-31-dc59516c3c64>:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric1 = load_metric("precision")


Saving model checkpoint to roberta_text_100_step/checkpoint-5
Configuration saved in roberta_text_100_step/checkpoint-5/config.json
Model weights saved in roberta_text_100_step/checkpoint-5/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4429
  Batch size = 8
Saving model checkpoint to roberta_text_100_step/checkpoint-10
Configuration saved in roberta_text_100_step/checkpoint-10/config.json
Model weights saved in roberta_text_100_step/checkpoint-10/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely igno

TrainOutput(global_step=45, training_loss=0.2582629097832574, metrics={'train_runtime': 1449.0853, 'train_samples_per_second': 2138.728, 'train_steps_per_second': 267.341, 'total_flos': 94719979929600.0, 'train_loss': 0.2582629097832574, 'epoch': 0.01})

testing accuracy  = 'test_accuracy': 0.9990956364458512

In [ ]:
model.save_pretrained("roberta_text_100_step_best")
result_3 = trainer.predict(tokenized_datasets["test"])
result_3

Configuration saved in roberta_text_100_step_best/config.json
Model weights saved in roberta_text_100_step_best/pytorch_model.bin
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 8846
  Batch size = 8


PredictionOutput(predictions=array([[-3.2348757,  3.0571947],
       [-3.5427687,  3.2835732],
       [ 3.4120438, -3.334492 ],
       ...,
       [-3.5680723,  3.2972438],
       [-3.325041 ,  3.131107 ],
       [ 3.4108064, -3.3507955]], dtype=float32), label_ids=array([1, 1, 0, ..., 1, 1, 0]), metrics={'test_loss': 0.005341419018805027, 'test_precision': 0.9990956364458512, 'test_recall': 0.9990956364458512, 'test_f1': 0.9990956364458512, 'test_accuracy': 0.9990956364458512, 'test_runtime': 297.3977, 'test_samples_per_second': 29.745, 'test_steps_per_second': 3.719})


# (IGNORE) political fake news classification with title content with RoBerta Model


In [ ]:
from transformers import RobertaTokenizer
from datasets import load_dataset
dataset_title = load_dataset("csv", data_files={"train":  "train_title_clean.csv", "test": "test_title_clean.csv","vali":"vali_title_clean.csv"})

import transformers
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
def tokenize_function(examples):
    return tokenizer(examples["title"], padding="max_length",truncation=True)

tokenized_datasets_title = dataset_title.map(tokenize_function, batched=True)

  0%|          | 0/3 [00:00<?, ?it/s]

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_

  0%|          | 0/32 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
MODEL_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_name)
config = AutoConfig.from_pretrained(MODEL_name)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_name)

from transformers.optimization import AdamW
import numpy as np
from datasets import load_metric
from sklearn.metrics import accuracy_score,recall_score,precision_score
from transformers import EarlyStoppingCallback, IntervalStrategy
def custom_metrics(eval_pred):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision = metric1.compute(predictions=predictions, references=labels, average="micro")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="micro")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="micro")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}


optimizer = AdamW(model.parameters(), lr=1e-3, correct_bias=True)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/h

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="roberta_title_100_step", save_total_limit = 5,weight_decay=0.1,evaluation_strategy = IntervalStrategy.STEPS,eval_steps = 100,save_steps = 100, num_train_epochs=100,load_best_model_at_end=True,metric_for_best_model = 'accuracy')
from transformers import EarlyStoppingCallback, IntervalStrategy

small_train_dataset = tokenized_datasets_title["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets_title["vali"].shuffle(seed=42)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=custom_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: title. If title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 31428
  Num Epochs = 100
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 392900
  Number of trainable parameters = 124647170


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,No log,0.016169,0.998218,0.998218,0.998218,0.998218
200,No log,0.039132,0.995546,0.995546,0.995546,0.995546
300,No log,0.049423,0.990646,0.990646,0.990646,0.990646
400,No log,0.037743,0.994655,0.994655,0.994655,0.994655


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: title. If title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4490
  Batch size = 8
<ipython-input-8-ed166d679248>:16: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric1 = load_metric("precision")
Saving model checkpoint to roberta_title_100_step/checkpoint-100
Configuration saved in roberta_title_100_step/checkpoint-100/config.json
Model weights saved in roberta_title_100_step/checkpoint-100/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: title. If title are not exp

TrainOutput(global_step=400, training_loss=0.05571948528289795, metrics={'train_runtime': 884.2969, 'train_samples_per_second': 3554.01, 'train_steps_per_second': 444.308, 'total_flos': 841955377152000.0, 'train_loss': 0.05571948528289795, 'epoch': 0.1})

In [ ]:
model.save_pretrained("roberta_title_100_step_best")
result_3 = trainer.predict(tokenized_datasets_title["test"])
result_3

Configuration saved in roberta_title_100_step_best/config.json
Model weights saved in roberta_title_100_step_best/pytorch_model.bin
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: title. If title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 8980
  Batch size = 8


PredictionOutput(predictions=array([[ 2.1895838, -2.426    ],
       [-4.6067085,  4.5801535],
       [ 2.179265 , -2.398108 ],
       ...,
       [-4.650122 ,  4.5940638],
       [ 2.2290864, -2.462246 ],
       [ 2.2557857, -2.4880028]], dtype=float32), label_ids=array([0, 1, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.015764491632580757, 'test_precision': 0.9983296213808464, 'test_recall': 0.9983296213808464, 'test_f1': 0.9983296213808464, 'test_accuracy': 0.9983296213808464, 'test_runtime': 274.4296, 'test_samples_per_second': 32.722, 'test_steps_per_second': 4.092})